# PDGFR Circle Heatmap

This notebooks takes the genes that are a hit from the Brca NCI-Nature_2016 (PDGFR-beta pathway) and maps them on a large circle heat map. 

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats
import re
import sys 
import statsmodels.stats.multitest

import gseapy as gp
from gseapy.plot import barplot, dotplot

import cptac
import cptac.utils as u
import plot_utils as p 

/Users/Lindsey/anaconda3/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


Load df with all of the genes that are FDR significant. Then get list of just the gene names and use them to run a GSEA. 

In [2]:
prot_FDR = pd.read_csv("../Step3.1_Pearson_dfs_by_cancer/csv_files/Brca_EGFR_all_pearson_FDR.csv")
df_FDR= prot_FDR.drop(['Unnamed: 0'], axis=1)
df_FDR = df_FDR.set_index("Comparison")
df1_transposed = df_FDR.T 
df1_transposed


Comparison,EGFR_proteomics,MPP6_proteomics,CPNE8_proteomics,CRYBG3_proteomics,PSAT1_proteomics,PHC3_proteomics,GATA3_proteomics,KRT16_proteomics,KRT5_proteomics,NXN_proteomics,...,MNS1_proteomics,RRP1_proteomics,ZDHHC20_proteomics,SFXN3_proteomics,AGAP3_proteomics,PPP1R14B_proteomics,RASA1_proteomics,HMGN1_proteomics,DDX27_proteomics,MSH6_proteomics
Correlation,1.0,6.482017e-01,6.164527e-01,6.099971e-01,6.093187e-01,-6.032044e-01,-5.989113e-01,5.953255e-01,5.947704e-01,5.928643e-01,...,0.270088,0.229647,0.229381,0.229358,-0.229335,0.229296,-0.229296,-0.229293,0.229265,0.229263
P_value,0.0,4.834438e-15,2.229263e-13,4.612176e-13,4.973595e-13,9.738761e-13,1.547846e-12,2.267369e-12,2.404384e-12,2.938573e-12,...,0.013535,0.013554,0.013665,0.013675,0.013685,0.013701,0.013701,0.013703,0.013714,0.013715
fdr_bh_p_val,0.0,2.441391e-11,7.505185e-10,1.004666e-09,1.004666e-09,1.639358e-09,2.233321e-09,2.514358e-09,2.514358e-09,2.514358e-09,...,0.049494,0.049545,0.049935,0.049953,0.049970,0.049980,0.049980,0.049980,0.049991,0.049991


In [3]:
brca_prot = df1_transposed.columns.values.tolist()
brca_genes = []
for gene in brca_prot :
    brca_genes.append((re.sub("_proteomics", "", gene)))
len(brca_genes)

2771

Run GSEA using reactome 2016 set

In [4]:
brca_enr = gp.enrichr(gene_list = brca_genes, description='Tumor_partition', gene_sets='NCI-Nature_2016', 
                       outdir='test/enrichr_kegg')
brca_enr.res2d.head(2)

,Gene_set,Term,Overlap,P-value,Adjusted P-value,Old P-value,Old Adjusted P-value,Odds Ratio,Combined Score,Genes
0,NCI-Nature_2016,Urokinase-type plasminogen activator (uPA) and...,25/42,3.300094e-12,6.897197e-10,0,0,4.448715,117.610996,ITGB1;ITGAM;LRP1;SRC;ITGB3;ITGB2;SERPINE1;PLG;...
1,NCI-Nature_2016,PDGFR-beta signaling pathway Homo sapiens c901...,48/128,5.660464e-12,5.915185e-10,0,0,2.802691,72.582722,USP6NL;DOCK4;TAGLN;LRP1;SRC;ARPC1B;ITGB3;BRK1;...


In [5]:
#get just the pdgfr genes
brca_df = brca_enr.res2d
pdgfr = brca_df.iloc[1,9]
pdgfr = pdgfr.split(';')
len(pdgfr)


48

In [6]:
#Get append version of the df with all cancer type, fdr sig trans results
df_FDR_append = pd.read_csv("../Step3.2_combining_pearson_dfs/csv_files/pancan_EGFR_pearson_sig_all_prot_append_FDR.csv")
df_FDR_append = df_FDR_append.drop(['Unnamed: 0'], axis=1)


In [7]:
#filter down df with just pdgfr genes 
pdgfr_column_names = []
for gene in pdgfr:
    gene += "_proteomics"
    pdgfr_column_names.append(gene)

df_FDR_pdgfr = df_FDR_append[df_FDR_append.Comparison.isin(pdgfr_column_names)]
df_FDR_pdgfr

,Comparison,Correlation,P_value,fdr_bh_p_val,Cancer Type
175,ARPC1B_proteomics,-0.431473,8.240555e-06,0.000500,GBM
194,HCK_proteomics,-0.426001,1.101720e-05,0.000605,GBM
206,ARPC3_proteomics,-0.423330,1.267190e-05,0.000655,GBM
299,ARPC2_proteomics,-0.406327,3.005095e-05,0.001072,GBM
416,YES1_proteomics,0.383762,8.813311e-05,0.002262,GBM
...,...,...,...,...,...
8605,SRC_proteomics,0.480798,1.389641e-07,0.000060,Lscc
8712,BAIAP2_proteomics,0.379219,5.180059e-05,0.004465,Lscc
8815,ITGAV_proteomics,0.342779,2.818073e-04,0.013723,Lscc
8916,JUN_proteomics,0.317601,8.099733e-04,0.027683,Lscc


In [9]:
#Make plot using plot utils
p.plotCircleHeatMap(df_FDR_pdgfr, "P_value","Correlation","Comparison","Cancer Type",plot_width= 1200, plot_height = 650)

NameError: name 'plotCircleHeatMap' is not defined